In [ ]:
pip install openai

In [ ]:
pip install git+https://github.com/openai/whisper.git 

In [ ]:
%conda install -c conda-forge ffmpeg

In [ ]:
!conda --version

In [ ]:
ffmpeg -version

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

In [ ]:
from openai import OpenAI

#openai.api_type = "openai"

client = OpenAI(api_key=os.getenv("OPENAI_WHISPER_API_KEY"))

# Open the audio file
with open("./audio/test.mp3", "rb") as audio_file:
    # Transcribe the audio file
    result = client.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file
    )

# Print the transcribed text
print(result.text)


In [ ]:
import whisper
import os
print(os.getcwd())

model = whisper.load_model("tiny.en")
os.chdir(os.getcwd())
print(os.getcwd())
file_path = r".\audio\presto.wav"
result = model.transcribe(file_path)
print(result["text"])

In [ ]:
import whisper

model = whisper.load_model("base")

# load audio and pad/trim it to fit 30 seconds
file_path = r".\audio\test.mp3"
audio = whisper.load_audio(file_path)
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)